# NumPy: Basic Operations

In [1]:
import numpy as np

## Basic Arithmetic Operations

### Elementwise operations (Not inplace)

Most basic arithmetic operators on arrays apply *elementwise*. A new array is created an filled with the result.

In [2]:
m1 = np.array([[1, 2, 3], 
               [2, 3, 4]])

m2 = np.array([[1, 4, -1], 
               [4, 3, 1]])

In [3]:
# Elementwise multiplication
# Calls np.multiply()
print(m1 * m2)

[[ 1  8 -3]
 [ 8  9  4]]


In [4]:
# Elementwise addition
# numpy.add()
print(m1 + m2)

[[2 6 2]
 [6 6 5]]


In [5]:
# Elementwise subtraction
# Calls np.subtract()
print(m1 - m2)

[[ 0 -2  4]
 [-2  0  3]]


In [6]:
# Elementwise division

# Calls np.divide() 
print(m1 / m2)

[[ 1.   0.5 -3. ]
 [ 0.5  1.   4. ]]


In [7]:
# Elementwise Floor divide
# Return the largest integer smaller or equal to the division of the inputs. 

# Calls np.floor_divide()
print(m1 // m2)

[[ 1  0 -3]
 [ 0  1  4]]


In [8]:
# Calls np.pow()
m1**5

array([[   1,   32,  243],
       [  32,  243, 1024]])

In [9]:
# Calls np.mod()
m1 % 2

array([[1, 0, 1],
       [0, 1, 0]])

### Elementwise operations (Inplace)

In [10]:
m = np.array([1,1])

# InPlace multiplication
m *= 2

print(m)

[2 2]


In [11]:
m = np.array([1,1])

# InPlace addition
m += 2

print(m)

[3 3]


In [12]:
m = np.array([1,1])

# InPlace subtraction
m -= 2

print(m)

[-1 -1]


In [13]:
# The data type is a float (!)
m = np.array([1., 1.])

# InPlace divison
m /= 2

print(m)

[0.5 0.5]


In [14]:
# The data type is an int (!)
m = np.array([1, 1])

# InPlace Floor divison
m //= 1

print(m)

[1 1]


**What if we divide an int32 by some value?**

This raises an error: `UFuncTypeError: Cannot cast ufunc 'divide' output from dtype('float64') to dtype('int64') with casting rule 'same_kind'`

In [15]:
# The data type is an int (!)
m = np.array([1, 1])

# Standard divison
m /= 1

print(m)

UFuncTypeError: Cannot cast ufunc 'divide' output from dtype('float64') to dtype('int64') with casting rule 'same_kind'

Since the `/=` calls the `divide()` method it is worth taking a look at divide's documentation.
https://numpy.org/doc/stable/reference/generated/numpy.divide.html

As can be seen in the documentation, divide provides a parameter `casting` which is detailed in https://numpy.org/doc/stable/reference/ufuncs.html#ufuncs-kwargs. Following the links in the documentation, we can see that the following values are support by `casting`:

- ‘no’ means the data types should not be cast at all.
- ‘equiv’ means only byte-order changes are allowed.
- ‘safe’ means only casts which can preserve values are allowed.
- ‘same_kind’ means only safe casts or casts within a kind, like float64 to float32, are allowed.
- ‘unsafe’ means any data conversions may be done.





In [16]:
# The data type is an int (!)
m = np.array([1, 1])

# Let's play around with the casting operator just to see how it works
# If we set it to unsafe, the method should not raise an error (which is obviously dangerous)
m = np.divide(m, 2, out=m, casting='unsafe')

print(m)

[0 0]


### Overflows and underflows

In the previous notebook, we have already seen that Python doesn't perform any type checks when we manipulate values in an array (see example below). But what if there is an overflow or underflow? 

In [17]:
m = np.array([1, 1])

# The floating point value becomes an integer
m[0] = 0.6

print(m)

[0 1]


Let's produce an overflow to see what happens ...

In [18]:
m = np.array([0, 0], dtype=np.uint8)

print(m)

# Causes an underflow
m -= 1

print(m)

[0 0]
[255 255]


In [19]:
m = np.array([128, 128], dtype=np.uint8)

print(m)

# Causes an overflow
m *= 2

print(m)

[128 128]
[0 0]


Note that no error is raise like in C! So watch out that you always use the correct type. <br/>
Sometimes it's advisable to add additional data type checks to functions if you are unsure that the passed argument has the correct data type.

For example:

In [20]:
m = np.array([1])
#m = np.array([1.])

if isinstance(m, np.ndarray) and not np.issubdtype(m.dtype, np.floating):
    raise TypeError('Invalid data type')

TypeError: Invalid data type

Since the documentation of `numpy.divide()` mentions that the `np.seterr()` can set whether to raise or warn on overflow, underflow and division by zero, you might be tempted to say "Ok, then I simply reconfigure numpy to do checks". However, be warned that that `np.seterr()` does only affect floating point operations!
  

In [25]:
# The following two lines only affect the behavior of floating point operations.
# np.seterr('ignore')
np.seterr('raise')

m = np.array([1], dtype=np.uint8)
#m = np.array([1.], dtype=np.float16)

# Error
m += 3000000

print(m)

[193]


So, always watch out for correct data types. For example, many image processing libraries return an uint8 matrix when loading an image. <br/>
If you forget to change the data type and apply arithmetic operations (in particular, if these operations are inplace) on the uint8 matrix, the consequences can be severe ...

### Broadcasting

The term broadcasting describes how NumPy treats arrays with different shapes during arithmetic operations. Subject to certain constraints, the smaller array is "broadcast" across the larger array so that they have compatible shapes.

As we have seen previously, if we multiply a matrix with some float value, the float value will be automatically broadcasted to the shape of the matrix. We will now look at some more advanced examples of broadcasting.

In [37]:
m1 = np.ones((5, 5, 3))
m2 = np.array([2, 2, 2])

print(m1.shape, m2.shape)

# Image  (3d array): 5 x 5 x 3
# Scale  (1d array):         3
# Result (3d array): 5 x 5 x 3
m = m1 * m2
print(m.shape)

(5, 5, 3) (3,)
(5, 5, 3)


In [38]:
m1 = np.ones((5, 7, 1, 3))
m2 = np.ones((7, 5, 1))

print(m1.shape, m2.shape)

# Image  (4d array): 5 x 7 x 1 x 3
# Scale  (3d array):     7 x 5 x 1
# Result (3d array): 5 x 7 x 5 x 3
m = m1 * m2
print(m.shape)

(5, 7, 1, 3) (7, 5, 1)
(5, 7, 5, 3)


### Matrix multiplication

Matrix multiplications (a.k.a. dot prodict) can be implemented with the `@` operator. Alternatively, we can use the `np.dot()` method.

In [40]:
m1 = np.arange(6).reshape(2, 3)
m2 = np.arange(12).reshape(3, 4)

# (2, 3) x (3, 4) =>  (2, 4)
m = m1 @ m2

print(m)
print(m.shape)

[[20 23 26 29]
 [56 68 80 92]]
(2, 4)


Or with `np.dot()` ...

In [41]:
m1 = np.arange(6).reshape(2, 3)
m2 = np.arange(12).reshape(3, 4)

# (2, 3) x (3, 4) =>  (2, 4)
m = np.dot(m1, m2)

print(m)
print(m.shape)

[[20 23 26 29]
 [56 68 80 92]]
(2, 4)
